In [ ]:
import numpy as np
import pandas as pd 
data = pd.read_csv('C:/Users/Gvaidyam/Desktop/Civis/individual_data.csv')

data["adult"] = (data["age"]>=18).astype(int)

perc_adult = sum(data["adult"])/data.shape[0]

ad_per_hh = np.sum(data["adult"])/len(pd.unique(data["hh_id"]))

chi_per_hh = (data.shape[0]-np.sum(data["adult"]))/len(pd.unique(data["hh_id"]))

perc_college_grad = np.sum(data["is_college_graduate"])/np.sum(data["adult"])

perc_college_grad_all = np.sum(data["is_college_graduate"])/(data.shape[0])

college_table = pd.crosstab(index=data["is_college_graduate"], 
                           columns=data["adult"])#.apply(lambda r: r/r.sum(), axis=1)
college_table.index= ["no","yes"]

data.loc[(data["adult"]==0) &(data["is_college_graduate"]==1),"is_college_graduate"]=0

data.loc[data["gender"]=="F","Gender"]="Female"
data.loc[data["gender"]=="FEMALE","Gender"]="Female"
data.loc[data["gender"]=="Female","Gender"]="Female"
data.loc[data["gender"]=="M","Gender"]="Male"
data.loc[data["gender"]=="MALE","Gender"]="Male"
data.loc[data["gender"]=="Male","Gender"]="Male"

data.drop('gender', axis=1, inplace=True)

data["gender_indicator"] = (data["Gender"]=="Male").astype(int)

adults_per_hh= pd.pivot_table(data,index=["hh_id"],values=["adult","gender_indicator"],aggfunc=np.sum).reset_index()


adults_per_hh.columns.values[2] = 'male_indicator'

data=pd.merge(data, adults_per_hh, on='hh_id', how='left')
data

def transform(dat):
    if  dat["married"] == "Single" and dat["adult_y"]>=2:
        return "Unmarried"
    if  dat["married"] == "Married" and dat["adult_y"]>=2:
        return "Married"
    if  dat["Gender"] == "Male" and dat["is_head_of_household"]==1 and dat["adult_y"]==1:
        return "single male"
    if  dat["Gender"] == "Female" and dat["is_head_of_household"]==1 and dat["adult_y"]==1:
        return "single female"



data_nokids=data[data["adult_x"]==1]
data_nokids["household_type"]=data_nokids.apply(transform,axis=1)
data_nokids1 = data_nokids[["hh_id","household_type"]].drop_duplicates()

data_nokids

data=pd.merge(data, data_nokids1, on='hh_id', how='left')

counts = data_nokids1 ["household_type"].value_counts()/np.sum(data_nokids1 ["household_type"].value_counts())

counts

household = pd.read_csv('C:/Users/Gvaidyam/Desktop/Civis/household_data.csv')

data=pd.merge(data, household, on='hh_id', how='left')

data_nokids1=pd.merge(data_nokids1, household, on='hh_id', how='left')

data_nokids1

import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="ticks")


# Draw a nested boxplot to show bills by day and sex
sns.boxplot(x="household_type", y="Household Income", data=data_nokids1, palette="PRGn")
sns.despine(offset=10, trim=True)
sns.plt.show()

data_nokids1.boxplot(column="Household Income",by="household_type")

data_nokids1.boxplot()
plt.show()

data_nokids1.dtypes

help(data_nokids1.describe())

bygroup_treatment = data_nokids1.groupby(['household_type'])

bygroup_treatment.mean()

ind = pd.read_csv("C:/Users/Gvaidyam/Desktop/Civis/individual_data.csv")
ind

cols=data_nokids.columns
cols

df_subset = data_nokids.reindex(columns=['hh_id', 'person_id', 'is_head_of_household', 'married', 'race', 'age',
       'voted_in_2012', 'is_college_graduate', 'adult_x', 'Gender', 'household_type'])

ind = pd.read_csv('C:/Users/Gvaidyam/Desktop/Civis/registration_status.csv')
ind

data_ind_all=pd.merge(df_subset, ind, on='person_id', how='left')
data_ind_all=pd.merge(data_ind_all, data_nokids1, on='hh_id', how='left')
data_ind_all["response"] = data_ind_all["is_registered_democrat"]
data_ind_all = data_ind_all.drop("is_registered_democrat", axis=1)


#preprocess data to convert categorical vars into numeric: 

cols = data_ind_all.columns
cols = cols.drop(['response'])
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in cols:
    data_ind_all.ix[:,col] = le.fit_transform( data_ind_all.ix[:,col])



train0 = data_ind_all.loc[data_ind_all["response"].isnull()==False ,]
test = data_ind_all.loc[data_ind_all["response"].isnull()==True ,]


train0

from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
kf = KFold(len(train), 5, shuffle=True, random_state=8)
lr = LogisticRegression(class_weight="balanced")
rf =  RandomForestClassifier(class_weight="balanced",min_samples_split=50,min_samples_leaf =20 )
gbm = GradientBoostingClassifier(random_state=1, n_estimators=100, max_depth=5)
clf = svm.SVC()

print(train.columns)

algorithms = [
    [GradientBoostingClassifier(random_state=1, n_estimators=25, max_depth=3), ['is_head_of_household','race','age','voted_in_2012','is_college_graduate','adult_x','Gender','household_type_x','Household Income','Is Urban Household'
                 ,'Is owner of home', 'Tercile of Census Tract']],
    [LogisticRegression(class_weight="balanced",random_state= 1), ['is_head_of_household','race','age','voted_in_2012','is_college_graduate','adult_x','Gender','household_type_x','Household Income','Is Urban Household'
                 ,'Is owner of home', 'Tercile of Census Tract']]]
    
kf = KFold(train0.shape[0], n_folds=3, random_state=1)
    
predictions = []
for train, test in kf:
    train_target =  train0["response"].iloc[train]
    full_test_predictions = []
    # Make predictions for each algorithm on each fold
    for alg, predictors in algorithms:
        # Fit the algorithm on the training data.
        alg.fit(train0[predictors].iloc[train,:], train_target)
        # Select and predict on the test fold.  
        # The .astype(float) is necessary to convert the dataframe to all floats and avoid an sklearn error.
        test_predictions = alg.predict_proba(train0[predictors].iloc[test,:].astype(float))[:,1]
        full_test_predictions.append(test_predictions)
    # Use a simple ensembling scheme -- just average the predictions to get the final classification.
    test_predictions = (full_test_predictions[0] + full_test_predictions[1]) / 2
    # Any value over .5 is assumed to be a 1 prediction, and below .5 is a 0 prediction.
    test_predictions[test_predictions <= .5] = 0
    test_predictions[test_predictions > .5] = 1
    predictions.append(test_predictions)

# Put all the predictions together into one array.
predictions = np.concatenate(predictions, axis=0)
accuracy = sum(predictions[predictions == train0["response"]]) / len(predictions)
print(accuracy)



# False positives.
fp_filter = (predictions == 1) & (train["response"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (train["response"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (train["response"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (train["response"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

acc = (tn + tp)/(tp+tn+fp+fn)

acc

from sklearn.cross_validation import cross_val_predict
features = train[['is_head_of_household','race','age','voted_in_2012','is_college_graduate','adult_x','Gender','household_type_x','Household Income','Is Urban Household'
                 ,'Is owner of home', 'Tercile of Census Tract']]
target = train["response"]
predictions = cross_val_predict(lr, features, target, cv=kf)
predictions = pd.Series(predictions)